### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

### CONSTANTES

In [3]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    ftm = str(today - relativedelta(days=1))
    itm = str((today - relativedelta(days=1)).replace(day=1))
    flm = str((today - relativedelta(days=1)).replace(day=1) - relativedelta(days=1))
    ilm = str(((today - relativedelta(days=1)).replace(day=1) - relativedelta(days=1)).replace(day=1))
else:
    itm = str(today.replace(day=1))
    ftm = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))
    flm = str(today.replace(day=1) - relativedelta(days=1))
    ilm = str((today.replace(day=1) - relativedelta(days=1)).replace(day=1))

In [4]:
# Me traigo el rr
rr = func_rr()

### QUERIES

In [5]:
q_ord = '''SELECT cn.country_name AS Country,
       COUNT(DISTINCT o.order_id) AS Orders,
       COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS Acq,
       COUNT(DISTINCT o.user.id) AS Active_Users,
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.order_status = 'CONFIRMED'
GROUP BY 1'''.format(itm,ftm)

q_m1 = '''WITH users_table AS (
    SELECT cn.country_name AS country,
           o.user.id AS user,
           SUM(o.is_acquisition) AS acq
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE o.registered_date BETWEEN DATE('{2}') AND DATE('{3}')
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2),
    reorder_table AS (
    SELECT ut.country AS country,
           ut.user AS user,
           ut.acq AS acq
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o 
    INNER JOIN users_table AS ut ON o.user.id = ut.user
    WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2,3)
SELECT ut.country AS Country,
       COUNT(DISTINCT CASE WHEN ut.acq > 0 THEN ut.user ELSE NULL END) AS Acq_LM,
       COUNT(DISTINCT CASE WHEN ut.acq = 0 THEN ut.user ELSE NULL END) AS Active_LM,
       COUNT(DISTINCT CASE WHEN ut.acq > 0 THEN rt.user ELSE NULL END) AS Acq_Reorder,
       COUNT(DISTINCT CASE WHEN ut.acq = 0 THEN rt.user ELSE NULL END) AS Active_Reorder
FROM users_table AS ut
LEFT JOIN reorder_table AS rt ON ut.country = rt.country AND ut.user = rt.user
GROUP BY 1'''.format(itm,ftm,ilm,flm)

q_trials = '''WITH orders_table AS (
    SELECT o.user.id AS user,
           o.registered_date AS fecha,
           bt.business_type_name AS business,
           CASE WHEN o.is_acquisition = 1 THEN TRUE ELSE FALSE END AS acq,
           cn.country_name AS country
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o 
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_business_type` AS bt ON o.business_type_id = bt.business_type_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE o.registered_date < CURRENT_DATE()
          AND o.order_status = 'CONFIRMED'),
    qc_table AS (
    SELECT ot.user AS user,
           ot.business AS business,
           ot.country AS country,
           ot.acq AS acq,
    FROM orders_table AS ot
    WHERE ot.business NOT IN ('Restaurant','Coffee','Courier','Courier Business')
          AND ot.fecha BETWEEN DATE('{0}') AND DATE('{1}')),
    history_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           SUM(CASE WHEN ot.business NOT IN ('Restaurant','Coffee','Courier','Courier Business') THEN 1 ELSE 0 END) AS qc
    FROM orders_table AS ot
    WHERE ot.fecha < DATE('{0}')
          OR ot.acq
    GROUP BY 1,2)
SELECT qct.country AS Country,
       COUNT(DISTINCT qct.user) AS Active_CQ,
       COUNT(DISTINCT CASE WHEN qct.acq = FALSE AND (ht.qc = 0 OR ht.qc IS NULL) THEN qct.user ELSE NULL END) AS Adq_QC
FROM qc_table AS qct
LEFT JOIN history_table AS ht ON qct.user = ht.user AND qct.country = ht.country
GROUP BY 1
ORDER BY 1'''.format(itm,ftm)

q_desired = '''WITH orders_table AS (
    SELECT o.registered_date AS fecha,
           cn.country_name AS country,
           o.order_id AS order_id,
           o.user.id AS user,
           IFNULL(o.shipping_amount_no_discount + o.amount_no_discount,0) AS total_paid,
           SUM(IFNULL(od.discount_amount,0)) AS total_disc
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
    UNNEST(discounts) AS od
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE o.registered_date BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(),INTERVAL -6 MONTH),MONTH) AND LAST_DAY(DATE_ADD(CURRENT_DATE(),INTERVAL -1 MONTH))
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2,3,4,5),
    users_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           COUNT(DISTINCT ot.order_id) AS orders,
           SAFE_DIVIDE(SUM(ot.total_disc),SUM(total_paid)) AS adr,
           MAX(ot.fecha) AS last_order,
           CASE WHEN MAX(ot.fecha) <= LAST_DAY(DATE_ADD(CURRENT_DATE(),INTERVAL -2 MONTH)) AND SAFE_DIVIDE(SUM(ot.total_disc),SUM(total_paid)) < 0.3 THEN 'Si' ELSE 'No' END AS desired
    FROM orders_table AS ot
    GROUP BY 1,2),
    returns_table AS (
    SELECT cn.country_name AS country,
           ut.user AS user,
           COUNT(DISTINCT o.order_id) AS orders
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    INNER JOIN users_table AS ut ON o.user.id = ut.user
    WHERE o.registered_date BETWEEN DATE_TRUNC(CURRENT_DATE(),MONTH) AND LAST_DAY(CURRENT_DATE())
          AND o.order_status = 'CONFIRMED'
          AND ut.desired = 'Si'
    GROUP BY 1,2)
SELECT ut.country AS Country,
       COUNT(DISTINCT ut.user) AS Desired,
       COUNT(DISTINCT rt.user) AS Return_Desired,
       SUM(rt.orders) AS Return_Desired_Orders
FROM users_table AS ut
LEFT JOIN returns_table AS rt ON ut.user = rt.user AND ut.country = rt.country
WHERE ut.desired = 'Si'
GROUP BY 1'''

q_crm = '''WITH wallet_table AS (
    SELECT DISTINCT wa.order_id AS order_id,
           d.campaignCode AS campaign,
           d.amount AS amount
    FROM `peya-bi-tools-pro.il_wallet.fact_wallet_attributions` AS wa,
    UNNEST (attributionDetails) AS d
    WHERE wa.operation = 'Purchase'),
    wc_table AS (
    SELECT cn.country_name AS country,
           o.registered_date AS fecha,
           wa.campaign AS campaign,
           COUNT(DISTINCT o.order_id) AS orders,
           COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS acq,
           SUM(CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN wa.amount / ce.rate_us ELSE 0 END) AS amount,
           SUM(CASE WHEN bi.payment_mode != 'TOTAL_AMOUNT' THEN wa.amount / ce.rate_us ELSE 0 END) AS amount_shopper
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    INNER JOIN `peya-bi-tools-pro.il_core.fact_order_funding` AS w ON o.order_id = w.order_id AND w.type IN (25,35)
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
    LEFT JOIN wallet_table AS wa ON o.order_id = wa.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
    WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2,3)
SELECT wct.country AS Country,
       wct.campaign AS Campaign,
       'WALLET' AS Benefit,
       SUM(wct.orders) AS Orders_CRM,
       SUM(wct.acq) AS Acq_CRM,
       SUM(wct.amount) AS Amount,
       SUM(wct.amount_shopper) AS Amount_Shopper
FROM wc_table AS wct
GROUP BY 1,2

UNION ALL

SELECT cn.country_name AS Country,
       tca.talon_campaign_name AS Campaign,
       'TALON COUPON' AS Benefit,
       COUNT(DISTINCT o.order_id) AS Orders_CRM,
       COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS Acq_CRM,
       SUM(CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS Amount,
       SUM(CASE WHEN bi.payment_mode != 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS Amount_Shopper
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
INNER JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON tc.talon_campaign_id = tca.talon_campaign_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.order_status = 'CONFIRMED'
GROUP BY 1,2'''.format(itm,ftm)

q_react = '''WITH orders_table AS (
    SELECT o.registered_date AS fecha,
           cn.country_name AS country,
           o.order_id AS order_id,
           o.user.id AS user
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
    UNNEST(discounts) AS od
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE o.registered_date <= CURRENT_DATE()
          AND o.order_status = 'CONFIRMED'),
    users_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           COUNT(DISTINCT ot.order_id) AS orders,
           MAX(ot.fecha) AS last_order,
           CASE WHEN MAX(ot.fecha) <= LAST_DAY(DATE_ADD(CURRENT_DATE(),INTERVAL -2 MONTH)) THEN 'Si' ELSE 'No' END AS reactivate
    FROM orders_table AS ot
    WHERE ot.fecha <= LAST_DAY(DATE_ADD(CURRENT_DATE(),INTERVAL -1 MONTH))
    GROUP BY 1,2),
    returns_table AS (
    SELECT ot.country AS country,
           ut.user AS user,
           COUNT(DISTINCT ot.order_id) AS orders
    FROM orders_table AS ot
    INNER JOIN users_table AS ut ON ot.user = ut.user
    WHERE ot.fecha BETWEEN DATE_TRUNC(CURRENT_DATE(),MONTH) AND LAST_DAY(CURRENT_DATE())
          AND ut.reactivate = 'Si'
    GROUP BY 1,2)
SELECT ut.country AS Country,
       COUNT(DISTINCT ut.user) AS Reactivated,
       COUNT(DISTINCT rt.user) AS Return_Reactivated,
       SUM(rt.orders) AS Return_Reactivated_Orders
FROM users_table AS ut
LEFT JOIN returns_table AS rt ON ut.user = rt.user AND ut.country = rt.country
WHERE ut.reactivate = 'Si'
GROUP BY 1'''

In [6]:
# Descargo la data
bq_ord = pd.io.gbq.read_gbq(q_ord, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 43.04rows/s]


In [7]:
bq_m1 = pd.io.gbq.read_gbq(q_m1, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 24.83rows/s]


In [8]:
bq_trials = pd.io.gbq.read_gbq(q_trials, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 41.97rows/s]


In [9]:
bq_desired = pd.io.gbq.read_gbq(q_desired, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 49.20rows/s]


In [10]:
bq_crm = pd.io.gbq.read_gbq(q_crm, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 32608/32608 [00:03<00:00, 8215.29rows/s]


In [11]:
bq_react = pd.io.gbq.read_gbq(q_react, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 43.07rows/s]


In [12]:
# Copio la data
orders = bq_ord.copy()
m1 = bq_m1.copy()
trials = bq_trials.copy()
desired = bq_desired.copy()
crm = bq_crm.copy()
react = bq_react.copy()

### TRABAJO

In [13]:
# Trabajo Orders
val = [i for i in orders.columns if i not in ['Country']]
orders[val] = orders[val].astype(float)
col = ['Country']
orders[col] = orders[col].apply(lambda x: x.astype(str).str.upper())
# Trabajo M1
val = [i for i in m1.columns if i not in ['Country']]
m1[val] = m1[val].astype(float)
col = ['Country']
m1[col] = m1[col].apply(lambda x: x.astype(str).str.upper())
# Trabajo Trials
val = [i for i in trials.columns if i not in ['Country']]
trials[val] = trials[val].astype(float)
col = ['Country']
trials[col] = trials[col].apply(lambda x: x.astype(str).str.upper())
# Trabajo Desired
val = [i for i in desired.columns if i not in ['Country']]
desired[val] = desired[val].astype(float)
col = ['Country']
desired[col] = desired[col].apply(lambda x: x.astype(str).str.upper())
# Trabajo CRM
val = [i for i in crm.columns if i not in ['Country','Campaign','Benefit']]
crm[val] = crm[val].astype(float)
col = ['Country','Campaign','Benefit']
crm[col] = crm[col].apply(lambda x: x.astype(str).str.upper())
crm['Campaign'] = crm['Campaign'].str.replace(' ', '')
# Trabajo React
val = [i for i in react.columns if i not in ['Country']]
react[val] = react[val].astype(float)
col = ['Country']
react[col] = react[col].apply(lambda x: x.astype(str).str.upper())

In [14]:
# Trabajo CRM
crm['Type'] = crm.apply(lambda x: filtros(x['Campaign'],'TIPOS',x['Benefit']),axis=1)
crm['Segment'] = crm['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
crm['Level'] = crm['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
crm['Campaña'] = crm['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
crm['Automated'] = crm['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
crm['Budget'] = crm['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
crm['Trial'] = crm['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
# Coloco el IVA
crm = func_iva(crm)
# Marco las campañas a filtrar
crm['Filtrar'] = crm['Campaign'].apply(filtrar_cam)
crm = crm[crm['Filtrar'] == 'No'].copy()
# Saco el IVA
used = 'Amount'
used_shopper = 'Amount_Shopper'
crm['Final'] = crm[used_shopper] / (1 + crm['IVA']) * crm['Extra'] + crm[used]

In [15]:
# Hago una PT de CRM
index = ['Country','Type','Budget','Trial']
values = ['Orders_CRM','Acq_CRM','Final']
final_crm = crm.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()

In [16]:
# Hago el merge para unir las tablas
final = orders.merge(m1,on=['Country'],how='outer').copy()
final = final.merge(trials,on=['Country'],how='outer').copy()
final = final.merge(desired,on=['Country'],how='outer').copy()
final = final.merge(react,on=['Country'],how='outer').copy()
final.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [17]:
# Creo las columnas necesarias
final['M1_Adq'] = final['Acq_Reorder'] / final['Acq_LM']
final['No_Return'] = (final['Active_LM'] - final['Active_Reorder']) / final['Active_LM']
final['%Trials'] = final['Adq_QC'] / final['Active_CQ']
final['%Return Desired'] = final['Return_Desired'] / final['Desired']
# Creo los RR
final['Orders RR'] = round(final['Orders'] / rr,0)
final['Acq RR'] = round(final['Acq'] / rr,0)
final['Adq_QC RR'] = round(final['Adq_QC'] / rr,0)
final['Reactivated RR'] = round(final['Return_Reactivated'] / rr,0)
final['Return_Desired RR'] = round(final['Return_Desired'] / rr,0)
final['RR'] = rr
final.replace([np.nan,np.inf,-np.inf],0,inplace=True)
final_crm['Orders_CRM RR'] = round(final_crm['Orders_CRM'] / rr)
final_crm['Acq_CRM RR'] = round(final_crm['Acq_CRM'] / rr)
final_crm['Final RR'] = round(final_crm['Final'] / rr)
final_crm['RR'] = rr
final_crm.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [18]:
# Hago un sort por Country
final.sort_values(['Country'],ascending=True,inplace=True)
final_crm.sort_values(['Country'],ascending=True,inplace=True)

### CARGA

In [19]:
carga(final,'1SRFsNmj6hx_P0p98u1jM7AQJp0EfaOLdiA-jpHnduLg','Crudo BC')

Carga Correcta!


In [20]:
carga(final_crm,'1SRFsNmj6hx_P0p98u1jM7AQJp0EfaOLdiA-jpHnduLg','Crudo CRM BC')

Carga Correcta!
